In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [3]:
class Descriptor:

    def __init__(self, name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)

    def __set__(self, instance, value):
        instance.__dict__[self.name] = value


class Typed(Descriptor):
    expected_type = type(None)

    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError(f'expected {self.expected_type!s}')

        super().__set__(instance, value)


class Unsinged(Descriptor):

    def __set__(self, instance, value):
        if value < 0:
            raise ValueError(f'expected >= 0 but got {value}')

        super().__set__(instance, value)


class MaxSized(Descriptor):

    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')

        super().__init__(name, **opts)

    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError(f'size must be < {self.size} but got {len(value)}')  # noqa: E501

        super().__set__(instance, value)


class Integer(Typed):
    expected_type = int


class UnsignedInteger(Integer, Unsinged):
    pass


class Float(Typed):
    expected_type = float


class UnsignedFloat(Float, Unsinged):
    pass


class String(Typed):
    expected_type = str


class SizedString(String, MaxSized):
    pass


class checkedmeta(type):

    def __new__(cls, clsname, bases, methods):
        for key, value in methods.items():
            if isinstance(value, Descriptor):
                value.name = key

        return type.__new__(cls, clsname, bases, methods)


class Stock(metaclass=checkedmeta):
    name = SizedString(size=8)
    shares = UnsignedInteger()
    price = UnsignedFloat()

    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price


"""

>>> s = Stock('ACME', 50, 91.1)
>>> s.name
'ACME'
>>> s.name = 'abcdefghijklm'
Traceback (most recent call last):
    ...
ValueError: size must be < 8 but got 13
>>> s.shares
50
>>> s.shares = -10
Traceback (most recent call last):
    ...
ValueError: expected >= 0 but got -10
"""

doctest.testmod()

TestResults(failed=0, attempted=5)

In [4]:
from functools import partial


class Descriptor:

    def __init__(self, name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)

    def __set__(self, instance, value):
        instance.__dict__[self.name] = value


def typed(expected_type, cls=None):
    if cls is None:
        return partial(typed, expected_type)

    super_set = cls.__set__

    def __set__(self, instance, value):
        if not isinstance(value, expected_type):
            raise TypeError(f'expected {value!s} but got {type(value)!s}')

        super_set(self, instance, value)

    cls.__set__ = __set__
    return cls


def unsigned(cls):
    super_set = cls.__set__

    def __set__(self, instance, value):
        if value < 0:
            raise ValueError(f'expected >= 0 but got {value}')

        super_set(self, instance, value)

    cls.__set__ = __set__
    return cls


def sized(cls):

    super_init = cls.__init__

    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')

        super_init(self, name, **opts)
    cls.__init__ = __init__

    super_set = cls.__set__

    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError(f'size must be < {self.size}'
                             f' but got {len(value)}')

        super_set(self, instance, value)
    cls.__set__ = __set__

    return cls


@typed(int)
class Integer(Descriptor):
    pass


@unsigned
class UnsignedInteger(Integer):
    pass


@typed(float)
class Float(Descriptor):
    pass


@unsigned
class UnsignedFloat(Float):
    pass


@typed(str)
class String(Descriptor):
    pass


@sized
class SizedString(String):
    pass


class Stock(metaclass=checkedmeta):
    name = SizedString(size=8)
    shares = UnsignedInteger()
    price = UnsignedFloat()

    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price


"""

>>> s = Stock('ACME', 50, 91.1)
>>> s.name
'ACME'
>>> s.name = 'abcdefghijklm'
Traceback (most recent call last):
    ...
ValueError: size must be < 8 but got 13
>>> s.shares
50
>>> s.shares = -10
Traceback (most recent call last):
    ...
ValueError: expected >= 0 but got -10
"""

doctest.testmod()

TestResults(failed=0, attempted=5)